In [ ]:
import pandas as pd
import networkx as nx

In [1]:
from iac_sketch import data, etl, sketch, transform, validate

In [2]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(etl)
importlib.reload(sketch)
importlib.reload(transform)
importlib.reload(validate)
# architect = sketch.Architect("../public/components")
# registry = architect.parse()

<module 'iac_sketch.validate' from '/Users/zhafen/repos/iac-sketch/iac_sketch/validate.py'>

In [3]:
import pandera.pandas as pa

In [4]:
extract_sys = etl.ExtractSystem()
registry = extract_sys.extract_entities("../public/components/*yaml")
registry

In [5]:
transform_sys = etl.TransformSystem()
registry = transform_sys.apply_preprocess_transforms(registry)
registry

In [6]:
reqs = registry.view(["requirement", "description", "status"])
reqs.query("value_status not in ['deployed']")

,value,value_description,value_status
entity,,,
accept_request,<NA>,We should be able to receive and assign a requ...,in design
archive_data_products,<NA>,We want to be able to archive some of a table'...,new
audit_cohorts,<NA>,Users must able to identify what patients were...,new
deliver_report_req,<NA>,"After completing the request, viewers should b...",new
formatted_correctly,<NA>,The manifest must be formatted correctly.,NaN
framework_manifest_data,<NA>,We need the decisions that make up the framewo...,in design
framework_manifest_documentation,<NA>,We need to convert the manifest data into usea...,new
fulfills_all_requirements,<NA>,All requirements must be met.,NaN
fully_connected,<NA>,No entities can exist in isolation. They must ...,NaN


In [ ]:
# Query links where entity is "accept_request", and repeat for each link

In [ ]:
validation_sys = validate.ValidationSystem()
invalid_reqs = validation_sys.validate_requirements(registry)
invalid_reqs

In [ ]:
validation_sys = validate.ValidationSystem()
invalid_comps = validation_sys.validate_components(registry)
invalid_comps

In [ ]:
registry.view("compinst")

In [ ]:
registry.view(["compdef", "metadata"]).query("~valid")

In [ ]:
transform_sys = etl.TransformSystem()
registry = transform_sys.apply_preprocess_transforms(registry)

In [ ]:
# Create a directed graph from the DataFrame
graph = nx.from_pandas_edgelist(
    registry["link"],
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.DiGraph,
)

# Visualize the graph
nx.draw(graph, with_labels=True)

In [ ]:
graph.add_nodes_from(registry["metadata"]["entity"].values)

In [ ]:
connected_components = [_ for _ in nx.connected_components(graph.to_undirected())]
metadata = registry["metadata"].set_index("entity")
metadata["connected_component_category"] = -1
for i, comps in enumerate(connected_components):
    metadata.loc[list(comps), "connected_component_category"] = i

In [ ]:
pair_connectivity = nx.all_pairs_node_connectivity(graph)

In [ ]:
sum(pair_connectivity["accept_request"].values()) > 0

In [ ]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

In [ ]:
# Find invalid testcases
registry.view(["testcase", "status"])

In [ ]:
import base64
import io, requests
from IPython.display import Image, display
from PIL import Image as im
import matplotlib.pyplot as plt

In [ ]:
def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    img = im.open(
        io.BytesIO(requests.get("https://mermaid.ink/img/" + base64_string).content)
    )
    plt.imshow(img)
    plt.axis("off")  # allow to hide axis

In [ ]:
graph = """
graph LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A
    E--> B & C & D
"""

In [ ]:
graphbytes = graph.encode("utf8")
base64_bytes = base64.urlsafe_b64encode(graphbytes)
base64_string = base64_bytes.decode("ascii")
result = requests.get("https://mermaid.ink/img/" + base64_string).content
parsed_result = io.BytesIO(result)
img = im.open(parsed_result)
img

In [ ]:
mm(
    """
flowchart LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A & E
    E--> B & C & D
"""
)